# "Optical Flow"

> "Understand the concept of optical flow"

- toc: true
- branch: master
- badges: false
- comments: true
- categories: [Computer Vision]
- hide: false
- search_exclude: false
- image: images/post-thumbnails/of.png
- metadata_key1: notes
- metadata_key2: 

In the previous blog post [here](https://ablearn.io/computer%20vision/2021/12/05/Image-Features-Understanding.html) we understand basics of "features" in an image. One of the most popular usecases for such tracking of features between 2 images would be detection motion of objects within the image.


# Optical Flow vs Motion Field

Let's try to understand the terms first. 

We take a object in the world (represented in the world coordinates) and project them onto a image plane. As the object moves in the world, we want to track it in the image plane.  This tracking is called *MOTION FIELD*

It is not possible to directly measure motion field since all we have in the image are brightness patterns. 

Using the variation in brightness patterns between consecutives images can help you track the object(s). This is called *OPTICAL FLOW*. 

OPTICAL FLOW indicates MOTION of objects in most of the cases. But there could be images with "illusions" that changes brightness patterns but does not correspond to "object movement" in them". For eg. see below. 

---
The object is moving but there is no change in brightness pattern. So very hard to detect motion

![](https://abhisheksreesaila.github.io/blog/images/stereo/op.png "Motion Field without Optical Flow")

Here the light source is moving and hence the brightness pattern is changing, but there is no motion. 

![](https://abhisheksreesaila.github.io/blog/images/stereo/mf.png "Optical Flow without Motion Field")



# Optical Flow Assumptions

Before we attempt to compute the optical flow. Lets assume 2 things

1. Brightness of a image point remains constant over time. Since we will be dealing with images taken in quick succession (in the order of milliseconds) this is a safe assumption. 

![](https://abhisheksreesaila.github.io/blog/images/stereo/opticalfloweq.png "Object moving in an image")

Mathemtically written as 

> $I(x+ \delta x, y+ \delta y, t+ \delta t) = I(x, y, t)$

2. Displacement ($ \delta x, \delta y $) is very small


# Optical Flow Estimation

We will look at the "classic" methods of optical flow estimation. OPENCV provides a lot of inbuilt methods to compute them

## Sparse Optical Flow

We compute optical flow by extracting important features only. We will use the "feature extraction" techniques from the [earlier blog post](https://ablearn.io/computer%20vision/2021/12/05/Image-Features-Understanding.html) and determine the optical flow. Sparse Optical Flow is fast but not very accurate.

The code is available here.

For this exercise we use the dataset from [KITTI Vision Car Dataset](http://www.cvlibs.net/datasets/kitti/) available here.

### Load Images

```python

 #Load two consecutive images
img1 = cv2.imread("images/000199_10.png")
img2 = cv2.imread("images/000199_11.png")

#Convert both images to grayscale
## We need the gray scale images for the functions below
gray1 = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)


```

### Feature Extraction

We will use the Shi-Tomasi corner detector from OPENCV. To understand how to define the parameters, you can use [this link](https://docs.opencv.org/master/dd/d1a/group__imgproc__feature.html#ga1d6bb77486c8f92d79c8793ad995d541) 


```python

feature_params = dict(maxCorners = 500, qualityLevel = 0.01, minDistance = 10,blocksize = 3)
prev = cv2.goodFeaturesToTrack(gray1,feature_params['maxCorners'], feature_params["qualityLevel"], feature_params["minDistance"], feature_params["blocksize"])


```

By obtaining the features and drawing them, we obtain the following similar to below.


![](https://abhisheksreesaila.github.io/blog/images/stereo/sparse-optical-flow-fe.png "Corners detected (features) extracted from the image")



### Run Optical Flow

Launch the Lucas Kanade Optical Flow algorithm (*cv2.calcOpticalFlowPyrLK*) with the features and both grayscale images.


```python


lk_params = dict(winSize = (15, 15) , maxLevel = 2 , criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# img1 and img2 = images
# prev = features of the first image
next, status, error = cv2.calcOpticalFlowPyrLK(img1, img2, prev, None, winSize=lk_params["winSize"], maxLevel=lk_params["maxLevel"],criteria=lk_params["criteria"])


#Store the Matches (status=1 means a match)
good_old = prev[status ==1]
good_new = next[status ==1]

```

### Visualize 

Go through each matched feature, and draw it on the second image

![](https://abhisheksreesaila.github.io/blog/images/stereo/opticalflowop1.png "Optical Flow Output")

Here is a video for a more intuitive show of the optical flow

> youtube: https://youtu.be/mNGbiAWzRSw


## Deep Optical Flow

In Deep Optical Flow we compute optical flow for every pixel. Just to clarify, we dont use deep learning (yet!) 

The full code is available here

1. Load the images using openv functions as before

###  Calculate Optical Flow

Wait! no feature extraction? Yes thats correct. Since we are runnning optical flow on every pixel, we can directly use the corresponding opencv function. If you'd like to understand how to tweak the parameters, you can visit this [link](https://docs.opencv.org/3.0-beta/modules/video/doc/motion_analysis_and_object_tracking.html#calcopticalflowfarneback)


```python

flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)

print("Shapes Gray & Flow")
print(gray1.shape)
print(flow.shape)
print(" ")

"""
Shapes Gray & Flow
(376, 1241)
(376, 1241, 2)
"""

```
> Important: Here flow is a 2d matrix which has the same shape as input but contains the "$ \delta x $" (represented by U) and "$ \delta y$" (represented by V) for each point (X,Y).  

![](https://abhisheksreesaila.github.io/blog/images/stereo/flow-shown.png "Flow Matrix Shown Visually")

Accorinding to OPENCV, we get U and V in cartesian coordinates. To show this visually it will be good if we can find "magnitude" and "direction".  then we will intuitively know how much it moved and in what angle. Catesian system does not provide that.  Luckily for us, "polar system coordinates" gives us what we want. 

```python

magnitude, angle = cv2.cartToPolar(flow[...,0],flow[...,1])

```

### Setting the HUE Value


### Converting HUE to RGB


### Video

> youtube: https://youtu.be/QkuOY7AObh8




# References

[Optical Flow Definition](https://www.youtube.com/watch?v=lnXFcmLB7sM&t=92s)

[Workshops from Think Autonomous Course](https://courses.thinkautonomous.ai/optical-flow)